# Imports

In [1]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
%matplotlib inline

# Loading the Datasets

In [3]:
titanic_test = pd.read_csv('data/test.csv')
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [4]:
titanic = pd.read_csv('data/train.csv')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
combined = [titanic, titanic_test]

# General purpose helpers

In [6]:
def fit_eval_model(fit_data, features, target, prd, scoring=None):
    """
        Wrapper to fit a sklearn predictor on a dataframe
        and optionally perform a diagnostic CV run.
        
        - fit_data is a dataframe including an Age column.
        - features is a list of columns in fit_data to use in
          fitting the model, it should _not_ include Age.
        - target is a string containing the name of the variable
          that is to be predicted.
        - prd is a predictor object.
        - scoring optional, the scoring method to be used in CV
          if not provided then CV is skipped  
    """
    
    if scoring:
        scores = cross_val_score(prd,
                                 fit_data.loc[:, features],
                                 fit_data[target], 
                                 scoring=scoring,
                                 cv=10)
        print("CV {:.8}: {:0.2f} (+/- {:0.2f})" \
              .format(scoring, scores.mean(), scores.std() * 2))


    prd.fit(fit_data.loc[:, features],
                   fit_data[target]);
    return prd

# Cleaning, Wrangling, Engineering

In this section I deal with missing data and engineering new features from the columns already present in the datasets.

The order of those processes is a bit of a mess because some engineered features depend on a feature with missing data that must be first imputated using other engineered features and so on.

## On missing data

### Training set
* 177(~20%) of **ages** are missing. Age is expected to be a strong predictor. I should work on remedying that, [Gertlowitz](http://gertlowitz.blogspot.com.br/2013/06/where-am-i-up-to-with-titanic-competion.html) used people's titles(Ms., Mr. etc.) to predict missing ages to good measure.

* 2(<1%) people are missing **embark** points. This shouldn't be too important, but my swarmplots indicated there might be something at work there.

* 687(~80%) people are missing **cabin** information. I expect deck location and floor to be a reasonable predictor, but there might not be a way to reliably estimate missing data.

### Test set
* 86(~20%) are missing **ages**.

* 1(<1%) is missing **fare**

* 327(~80%) are missing **cabin**.

## Feature Engineering I

### Titles

Following the work of Gertlowitz mentioned above I extract the titles out of people's names in order to exploit it for both age and survival estimation.

In [7]:
def get_titles (data):
    return data['Name'].str.extract(', (\w+).', expand=False)

In [8]:
for dataset in combined:
    dataset['Title'] = get_titles(dataset)

In [9]:
def group_titles (row):
    miss_titles = ['the', 'Ms', 'Mme', 'Mlle', 'Dona']
    mrs_titles = ['Lady']
    mr_titles = ['Jonkheer', 'Capt', 'Col', 'Don', 'Major', 'Sir', 'Rev']
    
    if row['Title'] in miss_titles:
        return 'Miss'
    elif row['Title'] in mr_titles:
        return 'Mr'
    elif row['Title'] in mrs_titles:
        return 'Mrs'
    elif row['Title'] == 'Dr':
        if row['Sex'] == 'male':
            return 'Mr'
        else:
            return 'Miss'
        
    else:
        return row['Title']

In [10]:
for dataset in combined:
    dataset['Title'] = dataset.apply(group_titles, 1)

### Deck

Another thing Gertlowitz did was extract the deck letter from cabin information.

In [11]:
def get_deck (data):
    return data.Cabin.str.extract('^(\w)', expand=False)

In [12]:
for dataset in combined:
    dataset['Cabin'] = dataset['Cabin'].fillna('Z')
    dataset['Deck'] = get_deck(dataset)

### Family Aboard

I think both Gertlowitz and Sehgal did this.

In [13]:
def get_nfam (data):
    return data.Parch + data.SibSp + 1

In [14]:
for dataset in combined:
    dataset['Fam'] = get_nfam(dataset)

### Alone

Another feature due to Sehgal

In [15]:
def get_alone (fam):
    return {True: 1, False: 0}[fam == 1]

In [16]:
for dataset in combined:
    dataset['Alone'] = dataset.Fam.apply(get_alone)

## Reencoding  the training set

For the used categorical features

In [17]:
def gen_fit_les (data):
    """
    Generates a dictionary of LabelEncoders with an entry
    for each column in data that is of type object
    """
    
    cat_cols_les = {x: LabelEncoder() \
                    for x in data if data[x].dtype.name == 'object'}

    for item in cat_cols_les.items():
        col = item[0]
        le = item[1]
        
        le.fit(data[col])
        
    return cat_cols_les

def les_transform (data, le_dict):
    """
    Transform data's columns with a dictionary generated by
    gen_fit_les (above)
    """
    
    for item in le_dict.items():
        col = item[0]
        le = item[1]
        
        data[col] = le.transform(data[col])

In [18]:
titanic.Embarked = titanic.Embarked.fillna('U')

# Because some labels are present in only one set, it's necessary
# to train the encoders on their concatenation
cat_cols_les = gen_fit_les(pd.concat([titanic, titanic_test]))
les_transform(titanic, cat_cols_les)

## Reencoding  the test set

In [19]:
titanic_test.Embarked = titanic_test.Embarked.fillna('U')

les_transform(titanic_test, cat_cols_les)

## Imputation of fares

For the one lone passenger in the test set without fare information I'm using the mean of the fares for his Pclass in the training set.

In [20]:
def imputate_fare(row):
    if np.isnan(row['Fare']):
        return titanic[titanic.Pclass == row['Pclass']] \
                .groupby('Ticket').mean()['Fare'].mean()
    else:
        return row['Fare']

In [21]:
titanic_test['Fare'] = titanic_test.apply(imputate_fare, 1)

## Imputation of ages

The age imputation models are trained on the whole set of data with valid ages, spanning both the training and test sets. Is that really kosher though?

In [22]:
age_features = ['Pclass', 'Title', 'Fam', 'Fare']

age_train = pd.concat([
                        titanic[~np.isnan(titanic.Age)],
                        titanic_test[~np.isnan(titanic_test.Age)]
                      ])

In [23]:
age_model = fit_eval_model(age_train, 
                            age_features,
                            'Age',
                            GradientBoostingRegressor(),
                            'neg_mean_absolute_error'
                           )

CV neg_mean: -8.42 (+/- 1.24)


In [24]:
titanic.loc[np.isnan(titanic.Age), 'Age'] = \
    age_model.predict(titanic[np.isnan(titanic.Age)] \
                       .loc[:, age_features])

In [25]:
titanic_test.loc[np.isnan(titanic_test.Age), 'Age'] = \
    age_model.predict(titanic_test[np.isnan(titanic_test.Age)] \
                       .loc[:, age_features])

## Age*Pclass

This is due to [Sehgal](https://www.kaggle.com/startupsci/titanic-data-science-solutions)

In [26]:
for dataset in combined:
    dataset['AgePclass'] = dataset['Age'] * dataset['Pclass']

# Training the model

Random forest classifier, just to get things going:

In [27]:
train_features = ['Title', 'Sex', 'AgePclass', 'Fare', 'Alone']

model = fit_eval_model(titanic, 
                       train_features,
                       'Survived',
                       GradientBoostingClassifier(),
                       'f1'
                      )

CV f1: 0.75 (+/- 0.13)


# Applying the model to the test set

In [28]:
predictions = model.predict(titanic_test.loc[:, train_features])

In [29]:
results = pd.DataFrame(titanic_test['PassengerId'])
results['Survived'] = predictions
results.to_csv('predictions.csv', 
                  columns=('PassengerId', 'Survived'), index=False)

# Experiments and stuff

## Feature importances for models

In [30]:
[*zip(age_features, age_model.feature_importances_)]

[('Pclass', 0.08876489124030235),
 ('Title', 0.17801058000031905),
 ('Fam', 0.14308991027674106),
 ('Fare', 0.5901346184826379)]

In [31]:
[*zip(train_features, model.feature_importances_)]

[('Title', 0.04633956606755527),
 ('Sex', 0.13003513934511465),
 ('AgePclass', 0.3226937624908486),
 ('Fare', 0.4622930621134703),
 ('Alone', 0.03863846998301115)]

## Model Selection

### For age imputation

In [32]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [33]:
def fit_age_model (reg):
    fit_eval_model(age_train, 
                   age_features, 
                   'Age',
                   reg,
                   'neg_mean_absolute_error'
                  )
    
def fit_age_model_multi (reg_list):
    for reg in reg_list:
        fit_age_model(reg)

In [34]:
model_list = [
              linear_model.LinearRegression(n_jobs=-1),
              DecisionTreeRegressor(),
              RandomForestRegressor(n_jobs=-1),
              svm.SVR(),
              neighbors.KNeighborsRegressor(n_jobs=-1),
              GradientBoostingRegressor(),
              ExtraTreesRegressor(n_jobs=-1),
]

fit_age_model_multi(model_list)

CV neg_mean: -8.88 (+/- 0.82)
CV neg_mean: -9.72 (+/- 1.40)
CV neg_mean: -8.92 (+/- 1.60)
CV neg_mean: -10.13 (+/- 0.81)
CV neg_mean: -9.77 (+/- 1.10)
CV neg_mean: -8.42 (+/- 1.25)
CV neg_mean: -9.37 (+/- 1.41)


Compared to a simple model that just assigns the median age of each title:

In [35]:
a = age_train.groupby('Title').median()

title_age_dict = {x: y for x,y in zip(a.index, a.Age)}

mae = 0
for idx, row in age_train.iterrows():
    mae += np.abs(title_age_dict[row['Title']] - row['Age'])

-mae / age_train.shape[0]

-9.574885277246654

From this:
* Most models seem to be equivalent under pairwise Welch t-tests.
* However I don't trust statistics and will favour Linear, RF and GBRT for further tuning, since they can perform better than the median model under best conditions.
* Most of the time a one-sample t-test can't distinguish between the median-based model and the RF model, but the GBRT and Linear models usually let me discard the null hypothesis.
* Ridge will probably be used as a surrogate for linear.

#### Hyperparameter tuning

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
rf_search = GridSearchCV(RandomForestRegressor(),
                         {
                             'criterion': ['mse', 'mae'],
                             'min_samples_split': [2, 4, 8, 16],
                             'min_samples_leaf': [1, 2, 4, 8, 16],
                             'n_estimators': [4, 8, 10, 16, 25]
                         },
                         n_jobs=-1
                        )
rf_search.fit(age_train.loc[:, age_features], age_train['Age']);

rf_best = rf_search.best_estimator_

In [39]:
fit_age_model(rf_best);

CV neg_mean: -8.47 (+/- 1.15)


In [40]:
gbt_search = GridSearchCV(GradientBoostingRegressor(),
                          {
                             'loss': ['ls', 'lad', 'huber'],
                             'min_samples_split': [2, 4, 8, 16],
                             'min_samples_leaf': [1, 2, 4, 8, 16],
                             'max_depth': [2, 3, 4, 5],
                             'n_estimators': [16, 64, 128, 256]
                          },
                          n_jobs=-1
                        )
gbt_search.fit(age_train.loc[:, age_features], age_train['Age']);

gbt_best = gbt_search.best_estimator_

In [ ]:
fit_age_model(gbt_best);

tl;dr either I chose the grid very badly or hyperparameter tuning is of little help here